In [9]:
import xarray as xr
import pandas as pd
import locale

# locale.setlocale(locale.LC_ALL,'en_US')
# locale.setlocale(locale.LC_ALL,'en_US')
dataset = xr.open_dataset('MiningProcess_Flotation_Plant_Database.h5', engine = 'h5netcdf')
df = dataset.to_pandas()


In [10]:
## Resampling
df = df.resample('1h').mean()

# # ##
# df.iloc[:-1, df.columns.get_loc('% Iron Feed')] = df['% Iron Feed'].iloc[1:].values
# df.iloc[:-1, df.columns.get_loc('% Silica Feed')] = df['% Silica Feed'].iloc[1:].values
# df.iloc[:-1, df.columns.get_loc('% Iron Concentrate')] = df['% Iron Concentrate'].iloc[1:].values
# df.iloc[:-1, df.columns.get_loc('% Silica Concentrate')] = df['% Silica Concentrate'].iloc[1:].values

# df = df.iloc[:-1]

## Split into input and output
X = df.iloc[:,:-2]
Y = df['% Silica Concentrate']

In [11]:
# Adding lagged variables

import numpy as np
X_copy = X.copy()
lagged_variables = {}
for lag in range(1,7):
    lagged_variable = np.zeros(len(X_copy))
    lagged_variable[lag:] = Y.iloc[:-lag]

    lagged_variables[f'% Silica Concentrate Lag {lag}'] = lagged_variable
    # X.loc[:,] = lagged_variable


for variable in X_copy.columns:
    X_variable = X_copy[variable]
    for lag in range(1,7):
        lagged_variable = np.zeros(len(X_variable))
        lagged_variable[lag:] = X_variable.iloc[:-lag]


        # X.loc[f'{variable} Lag {lag}'] = lagged_variable
        lagged_variables[f'{variable} Lag {lag}'] = lagged_variable

df_lagged_variables = pd.DataFrame(lagged_variables,index=X_copy.index)

X = X.join(df_lagged_variables)
X = X[5:]

In [12]:
remove_variables = False
# N = 42
if remove_variables:
    variables = np.load('variables.npy')
    X = X.iloc[:,variables]

In [13]:
X.shape

(4410, 153)

In [14]:
# Split to calibration-validation set and test set
X_cal = X['20170615':'20170723']
Y_cal = Y['20170615':'20170723']
X_test = X['20170412':'20170506']
Y_test = Y['20170412':'20170506']

In [15]:
# Remove missing values in between
# X_cal.loc['201707060400':'201707062200'] = pd.NA
# X_cal.dropna(inplace=True)

# Y_cal.loc['201707060400':'201707062200'] = pd.NA
# Y_cal.dropna(inplace=True)

X_test.loc['20170421':'20170424'] = pd.NA
X_test.dropna(inplace=True)

Y_test.loc['20170421':'20170424'] = pd.NA
Y_test.dropna(inplace=True)

C:\Users\antto\AppData\Local\Temp\ipykernel_13748\695225842.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.dropna(inplace=True)


In [16]:
# Store data

with pd.HDFStore('MiningProcess_Flotation_Plant_Database_train_split.h5') as store:
    store.put('X_cal', X_cal)
    store.put('Y_cal', Y_cal)
    store.put('X_test', X_test)
    store.put('Y_test', Y_test)